Study the relation ship of:
input: The percentage of current price compared to x days simple moving average
output: The proc

In [1]:
import pandas as pd
import talib

Read data

In [2]:
stock_data_store = pd.HDFStore('stock_day.h5')

In [3]:
symbols_frame = pd.read_csv("stock_symbols.csv")

In [4]:
symbols_frame.head()

,symbols
0,A
1,AA
2,AAAP
3,AAAU
4,AABA


In [5]:
symbol_list = symbols_frame.loc[:, "symbols"].tolist()

In [6]:
symbol_list[:5]

['A', 'AA', 'AAAP', 'AAAU', 'AABA']

In [7]:
filter_symbol_list= []
finished_number = 0
for symbol in symbol_list:
    symbol_frame = stock_data_store.select('day', where=['symbol=="{}"'.format(symbol)])
    if symbol_frame.iloc[-1, :].loc["adj_volume"] > 3000000:
        filter_symbol_list.append(symbol)
    finished_number += 1
    if finished_number%500 == 0:
        print("process: {}/{}".format(finished_number, len(symbol_list)))
        

process: 500/14370
process: 1000/14370
process: 1500/14370
process: 2000/14370
process: 2500/14370
process: 3000/14370
process: 3500/14370
process: 4000/14370
process: 4500/14370
process: 5000/14370
process: 5500/14370
process: 6000/14370
process: 6500/14370
process: 7000/14370
process: 7500/14370
process: 8000/14370
process: 8500/14370
process: 9000/14370
process: 9500/14370
process: 10000/14370
process: 10500/14370
process: 11000/14370
process: 11500/14370
process: 12000/14370
process: 12500/14370
process: 13000/14370
process: 13500/14370
process: 14000/14370


In [8]:
len(filter_symbol_list)

1724

In [9]:
filter_symbols_frame = pd.DataFrame({"symbols":filter_symbol_list})
filter_symbols_frame.to_csv("filter_stock_symbols.csv", index=False)

In [10]:
filter_symbol_list = pd.read_csv("filter_stock_symbols.csv").loc[:, "symbols"].tolist()

In [31]:
def get_symbol_test_data(param_set, symbol):

    symbol_frame = stock_data_store.select('day', where=['symbol=="{}"'.format(symbol)])
    close = symbol_frame.loc[:, "adj_close"]
    sma = talib.SMA(close, timeperiod=param_set["sma_period"])
    input_column = (close - sma)/close * 100

    output_column = (close.shift(param_set["out_shift_period"]) - close)/close * 100

    test_data = pd.concat([input_column, output_column], axis=1)

    test_data.columns = ["input", "output"]

    test_data.dropna(inplace=True)
    #test_data = test_data[test_data["input"] > 1]
    daily_change = abs((close - close.shift(1))/close).dropna()
    daily_change_mean= daily_change.mean()

    #test_data
    test_data = test_data[abs(test_data["input"]) > daily_change_mean * 100 * 0.5]
    return test_data

In [32]:
test_data = pd.DataFrame()

In [33]:
final_result_dict = {"sma_period":[], "out_shift_period":[], "correlation":[], "accurate_ratio":[] }

In [60]:
for sma_period in range(2 , 21):
    for out_shift_period in range(-10, 0):
        final_result_dict["sma_period"].append(sma_period)
        final_result_dict["out_shift_period"].append(out_shift_period)
        param_set = {"sma_period": sma_period, "out_shift_period": out_shift_period }
        param_set_number = 0
        symbol_test_data_list = []
        for symbol in filter_symbol_list:
            symbol_test_data = get_symbol_test_data(param_set, symbol)
            symbol_test_data_list.append(symbol_test_data)

            param_set_number += 1

        correlation = test_data.loc[:, "input"].corr(test_data.loc[:, "output"])
        final_result_dict["correlation"].append(correlation)
        
        test_data.loc[:, "output"]
        
        product = (test_data.loc[:, "input"] * test_data.loc[:, "output"]) > 0
        accurate_ratio = len(product[product>0])/len(product)
        final_result_dict["accurate_ratio"].append(accurate_ratio)
        test_data.to_csv(os.path.join("test_data", "sma_period_{}_out_shift_period_{}".format(sma_period, out_shift_period) + ".csv"))
        print("process: {}/{}".format(param_set_number, 20 * 10))

KeyError: 'accurate_ratio'